In [2]:
import pandas as pd
import numpy as np
import torch
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from datetime import datetime
import gc

# 1. 환경 설정 및 데이터 로드 (91개 피처 로직 완벽 유지)
DEVICE = "mps" if torch.backends.mps.is_available() else "cpu"

train_data = pd.read_csv('../../data/raw/train.csv')
test_data = pd.read_csv('../../data/raw/test_x.csv')

train_data = train_data.drop(train_data[train_data.familysize > 50].index)

drop_list = ['QaE', 'QbE', 'QcE', 'QdE', 'QeE', 'QfE', 'QgE', 'QhE', 'QiE', 'QjE',
             'QkE', 'QlE', 'QmE', 'QnE', 'QoE', 'QpE', 'QqE', 'QrE', 'QsE', 'QtE',
             'index', 'hand']
replace_dict = {'education': str, 'engnat': str, 'married': str, 'urban': str}

train_y = 2 - train_data['voted'].to_numpy() # voted 1->1, 2->0
train_x = train_data.drop(drop_list + ['voted'], axis=1).astype(replace_dict)
test_x = test_data.drop(drop_list, axis=1).astype(replace_dict)

train_x = pd.get_dummies(train_x)
test_x = pd.get_dummies(test_x)

# 수동 스케일링 (0.78116 성공 로직)
train_x_np = train_x.to_numpy().astype(float)
test_x_np = test_x.to_numpy().astype(float)

train_x_np[:, :20] = (train_x_np[:, :20] - 3.) / 2.
test_x_np[:, :20] = (test_x_np[:, :20] - 3.) / 2.
train_x_np[:, 20] = (train_x_np[:, 20] - 5.) / 4.
test_x_np[:, 20] = (test_x_np[:, 20] - 5.) / 4.
train_x_np[:, 21:31] = (train_x_np[:, 21:31] - 3.5) / 3.5
test_x_np[:, 21:31] = (test_x_np[:, 21:31] - 3.5) / 3.5

# 2. TabNet 설정 및 학습 (스케줄러 오류 수정)
N_SKFOLD = 5
skf = StratifiedKFold(n_splits=N_SKFOLD, shuffle=True, random_state=42)

final_predictions = np.zeros(len(test_x_np))
fold_aucs = []

for fold, (train_idx, valid_idx) in enumerate(skf.split(train_x_np, train_y)):
    X_train, y_train = train_x_np[train_idx], train_y[train_idx]
    X_valid, y_valid = train_x_np[valid_idx], train_y[valid_idx]
    
    # TabNet 모델 정의 (StepLR 사용으로 'metrics' 에러 원천 봉쇄)
    clf = TabNetClassifier(
        n_d=32, n_a=32, n_steps=3, # n_steps를 조금 줄여 일반화 성능 강화
        gamma=1.3, lambda_sparse=1e-3,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=1e-2),
        scheduler_fn=torch.optim.lr_scheduler.StepLR, # 'metrics' 인자가 필요 없는 안전한 스케줄러
        scheduler_params=dict(step_size=10, gamma=0.9),
        mask_type='sparsemax',
        device_name=DEVICE
    )
    
    clf.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_valid, y_valid)],
        eval_name=['valid'],
        eval_metric=['auc'],
        max_epochs=100, 
        patience=15,
        batch_size=1024, virtual_batch_size=128,
        num_workers=0,
        drop_last=False
    )
    
    valid_preds = clf.predict_proba(X_valid)[:, 1]
    auc_score = roc_auc_score(y_valid, valid_preds)
    fold_aucs.append(auc_score)
    print(f"Fold {fold+1} Validation AUC: {auc_score:.5f}")
    
    # 0.78116 방식의 확률값 누적
    final_predictions += clf.predict_proba(test_x_np)[:, 1] / N_SKFOLD
    
    del clf; gc.collect()

# 3. 결과 출력 및 '2 - 확률' 로직 적용 제출
mean_auc = np.mean(fold_aucs)
print("\n" + "="*40)
print(f"최종 평균 Validation AUC: {mean_auc:.5f}")
print("="*40)

submission = pd.read_csv('../../data/raw/sample_submission.csv')

# [핵심] 0.78116의 '2. - sigmoid' 결과를 재현하기 위한 변환
# TabNet의 1(voted) 확률이 1에 가까울수록 결과는 1(voted)에 가깝게, 0에 가까울수록 2(not voted)에 가깝게 만듭니다.
submission['voted'] = 2.0 - final_predictions 

filename = f"./submissions/TabNet_Fixed_{datetime.now().strftime('%m%d-%H%M')}_AUC_{mean_auc:.4f}.csv"
submission.to_csv(filename, index=False)
print(f"제출 파일이 생성되었습니다: {filename}")

/opt/anaconda3/envs/voteai/lib/python3.11/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : mps
  warnings.warn(f"Device used : {self.device}")
/opt/anaconda3/envs/voteai/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


epoch 0  | loss: 0.73121 | valid_auc: 0.67094 |  0:00:03s


/opt/anaconda3/envs/voteai/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


epoch 1  | loss: 0.63003 | valid_auc: 0.72127 |  0:00:06s


/opt/anaconda3/envs/voteai/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


...

epoch 33 | loss: 0.51646 | valid_auc: 0.75162 |  0:01:40s


/opt/anaconda3/envs/voteai/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


epoch 34 | loss: 0.51472 | valid_auc: 0.75433 |  0:01:43s

Early stopping occurred at epoch 34 with best_epoch = 19 and best_valid_auc = 0.76093


/opt/anaconda3/envs/voteai/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


TypeError: Cannot convert a MPS Tensor to float64 dtype as the MPS framework doesn't support float64. Please use float32 instead.